In [177]:
%load_ext autoreload
%autoreload 2
    
import os
from pathlib import Path
import pandas as pd

import torch
from models.gaze_model import FineTuneModel, FaceModel, EyesModel, FaceGridModel
from dataset.dataset import GazeDetectionDataset
from facemesh import FaceMeshBlock, FaceMesh
from pupil_detection import IrisLM, IrisBlock
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_absolute_percentage_error as mape
import numpy as np
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
def train(model: FineTuneModel, dataloader_train: DataLoader):
    total_loss = 0.0
    model.train()
    preds_list = []
    labels_list = []
    for i, data in enumerate(dataloader_train):
        inputs, labels, inputs_eye_l, inputs_eye_r, inputs_mask = data['image'], data['coordinates'], \
                                                     data['eye_l'], data['eye_r'], data['face_mask']

        optimizer.zero_grad()

        outputs = model(inputs, inputs_eye_l, inputs_eye_r, inputs_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        preds_list += outputs.cpu().detach().tolist()
        labels_list += labels.cpu().detach().tolist()
        total_loss += loss.cpu().item()

    loss = total_loss / (i + 1)
    mape_value = mape(labels_list, preds_list)
    return loss, mape_value

In [179]:
def eval(model: FineTuneModel, dataloader_val: DataLoader):
    total_loss = 0.0
    model.eval()
    preds = []
    preds_list = []
    labels_list = []
    for i, data in enumerate(dataloader_val):
        inputs, labels, inputs_eye_l, inputs_eye_r, inputs_mask = data['image'], data['coordinates'], \
                                                     data['eye_l'], data['eye_r'], data['face_mask']
        with torch.no_grad():
            outputs = model(inputs, inputs_eye_l, inputs_eye_r, inputs_mask)
        loss = criterion(outputs, labels)
        total_loss += loss.cpu().item()
        preds_list += outputs.cpu().detach().tolist()
        labels_list += labels.cpu().detach().tolist()
        
    loss = total_loss / (i + 1)
    mape_value = mape(labels_list, preds_list)
    return loss, mape_value

In [180]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [181]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [182]:
base_folder = "./real_experiment/calibration_dataset/"
frames_folder = "frames"
annotations_folder = "annotations"
frames_folders_path = os.path.join(base_folder, frames_folder)
dfs = []
for frames_name in tqdm(os.listdir(frames_folders_path)):
    ann_path = os.path.join(base_folder, annotations_folder, f"{frames_name}.txt").replace("frames", "points")
    frames_path = os.path.join(frames_folders_path, frames_name)
    p = Path(frames_path).glob('*.png')
    paths = [str(path.absolute()) for path in p]
    df_files = pd.DataFrame({"paths": paths})
    df_files["ind"] = df_files.paths.apply(lambda x: Path(x).stem)
    df = pd.read_csv(
        ann_path,
        sep = " ",
        header=None
    )
    cols = [
        "timestamp", "x_gt", "y_gt", "x1", "y1",
        "x2", "y2", "screen_w", "screen_h"
    ]
    df.columns = cols
    
    df["x_normalized"] = df["x_gt"] / df["screen_w"]
    df["y_normalized"] = df["y_gt"] / df["screen_h"]
    df["timestamp"] = df["timestamp"].apply(str)
    full_df = df_files.merge(df, left_on="ind", right_on="timestamp").drop(columns = ["ind"])
    dfs.append(full_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 108.65it/s]


In [67]:
# frames_folder = "./real_experiment/calibration_dataset/frames/"
# p = Path(frames_folder).glob('*.png')
# paths = [str(path.absolute()) for path in p]
# df_files = pd.DataFrame({"paths": paths})
# df_files["ind"] = df_files.paths.apply(lambda x: Path(x).stem)

In [68]:
# df = pd.read_csv(
#     "./real_experiment/points_train.txt",
#     sep = " ",
#     header=None
# )
# cols = [
#     "timestamp", "x_gt", "y_gt", "x1", "y1",
#     "x2", "y2", "screen_w", "screen_h"
# ]
# df.columns = cols

# df["x_normalized"] = df["x_gt"] / df["screen_w"]
# df["y_normalized"] = df["y_gt"] / df["screen_h"]
# df["timestamp"] = df["timestamp"].apply(str)
# df.head()

In [69]:
# full_df = df_files.merge(df, left_on="ind", right_on="timestamp").drop(columns = ["ind"])

In [183]:
full_df = pd.concat(dfs)

In [184]:
full_df.shape

(855, 12)

In [185]:
full_df["participant_name"] = full_df["paths"].apply(lambda x: x.split("/")[-2].split("_")[2])

In [186]:
full_df_train = full_df[full_df["participant_name"] != "marina"]

In [187]:
test_diff_person = full_df[full_df["participant_name"] == "marina"]

In [188]:
test_diff_person, val_df_hard = train_test_split(test_diff_person, test_size = 0.3, random_state=42, shuffle=False)

In [189]:
full_df_train = pd.read_csv("real_experiment/calibration_dataset/cleaned_train.csv")

In [190]:
full_df_train.head()

,paths,timestamp,screen_w,screen_h,participant_name,...,y1,x2,y2,x_normalized,y_normalized
0,/home/ubuntu/projects/tweakle/gaze_detection/r...,1.697561e+09,2474,1520,misha,...,1045.666667,1010.333333,1205.666667,0.376044,0.740570
1,/home/ubuntu/projects/tweakle/gaze_detection/r...,1.697561e+09,2474,1520,misha,...,1342.500000,2451.500000,1497.000000,0.960287,0.935855
2,/home/ubuntu/projects/tweakle/gaze_detection/r...,1.697561e+09,2474,1520,misha,...,386.333333,978.666667,546.333333,0.363244,0.306798
3,/home/ubuntu/projects/tweakle/gaze_detection/r...,1.697561e+09,2474,1520,misha,...,1125.000000,790.500000,1285.000000,0.287187,0.792763
4,/home/ubuntu/projects/tweakle/gaze_detection/r...,1.697561e+09,2474,1520,misha,...,909.000000,2221.666667,1069.000000,0.865670,0.650658


In [272]:
NUM_SAMPLES = None
BATCH_SIZE = 1024
LEARNING_RATE = 1e-4
REDUCE_FACTOR = 0.5
PATIENCE = 3
NUM_EPOCHS = 30
WEIGHT_DECAY = 1e-4
CHECKPOINTS_PATH = "./checkpoints/"
EXPERIMENT_NAME = "calibration_unfreezed_guys_cleaned_weighted_loss_more_warmup_aug_tune_tune_tune"
LOSS_WEIGHTS = [0.3, 0.7]

In [273]:
train_df, test_df = train_test_split(full_df_train.head(NUM_SAMPLES), test_size = 0.1, random_state=42, shuffle=True)
train_df, val_df = train_test_split(full_df_train, test_size = 0.1, random_state=42, shuffle=True)

In [274]:
augmentations = A.Compose(
    [
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        A.Blur(p=0.3),
        A.CLAHE(p=0.1),
        A.RandomGamma(p=0.5),
        A.ImageCompression(quality_lower=75, p=0.5),
        A.MotionBlur(p=0.5)
    ]
)

In [275]:
trans_list = [A.Resize(192, 192)]

In [276]:
trans_list = [A.Resize(192, 192)]
dataset_train = GazeDetectionDataset(data = train_df, transform_list=trans_list,
                                     to_tensors=True, device=device, screen_features=False,
                                     transform=augmentations, augmentation_factor = 7)
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=0)
dataset_val = GazeDetectionDataset(data = val_df, transform_list=trans_list,
                                   to_tensors=True, device=device, screen_features=False)
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=0)
dataset_val_hard = GazeDetectionDataset(data = val_df_hard, transform_list=trans_list,
                                   to_tensors=True, device=device, screen_features=False)
dataloader_val_hard = DataLoader(dataset_val_hard, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=0)

Fusing layers... 
Fusing layers... 
Fusing layers... 


In [277]:
class RMSELoss(nn.Module):
    def __init__(self, device = "cuda:0", eps=1e-16, weights = None):
        super().__init__()
        self.eps = eps
        self.weights = weights
        self.device = device

    def __mse_loss(self, input, target, weights):
        if weights is not None:
            weights_tensor = torch.from_numpy(np.array(weights)).to(device)
            return torch.sum(weights_tensor * (input - target) ** 2) / weights_tensor.sum()
        return torch.mean((input - target) ** 2)
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.__mse_loss(yhat, y, self.weights) + self.eps)
        return loss

In [278]:
class EyesModel(nn.Module):
    def __init__(self, pretrained_model_eyes: nn.Module):
        super(EyesModel, self).__init__()
        self.backbone = pretrained_model_eyes.backbone
        self.regression_head_eyes = nn.Sequential(
            IrisBlock(128, 128), IrisBlock(128, 128),
            IrisBlock(128, 128, stride=2),
            IrisBlock(128, 128), IrisBlock(128, 128),
            IrisBlock(128, 128, stride=2),
            IrisBlock(128, 128), IrisBlock(128, 128),
        )
        # connect eyes
        self.fc = nn.Sequential(
            nn.Linear(2 * 128 * 1 * 1, 128),
            nn.ReLU(inplace=True),
            # nn.Linear(128, 128),
            # nn.ReLU(inplace=True),
        )

    def forward(self, x_eye_l, x_eye_r):
        x_eye_l = self.backbone(x_eye_l)
        x_eye_l = self.regression_head_eyes(x_eye_l)
        x_eye_l = x_eye_l.view(-1, 128 * 1 * 1)

        x_eye_r = self.backbone(x_eye_r)
        x_eye_r = self.regression_head_eyes(x_eye_r)
        x_eye_r = x_eye_r.view(-1, 128 * 1 * 1)
        x = torch.cat([x_eye_l, x_eye_r], 1)
        x = self.fc(x)
        return x

In [279]:
class FaceModel(nn.Module):
    def __init__(self, pretrained_model_face: nn.Module):
        super(FaceModel, self).__init__()
        self.backbone = pretrained_model_face.backbone
        self.regression_head_face = nn.Sequential(
            FaceMeshBlock(128, 128, stride=2),
            FaceMeshBlock(128, 128),
            FaceMeshBlock(128, 128),
            # FaceMeshBlock(128, 128),
            # FaceMeshBlock(128, 128),
            nn.Conv2d(128, 32, 1),
            nn.PReLU(32),
            FaceMeshBlock(32, 32),
        )
        self.fc = nn.Sequential(
            nn.Linear(32 * 3 * 3, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
        )

    def _preprocess(self, x):
        return x.to(torch.float32) * 2.0 - 1.0

    def forward(self, x_face):
        x_face = self._preprocess(x_face)
        x_face = nn.ReflectionPad2d((1, 0, 1, 0))(x_face)
        x_face = self.backbone(x_face)
        x_face = self.regression_head_face(x_face)
        x_face = x_face.view(-1, 32 * 3 * 3)
        x = self.fc(x_face)
        return x

In [280]:
class FineTuneModel(nn.Module):
    def __init__(
        self,
        pretrained_model_face: nn.Module,
        pretrained_model_eyes: nn.Module,
        screen_features: bool = False,
    ):
        super(FineTuneModel, self).__init__()
        self.face_model = FaceModel(pretrained_model_face)
        self.eyes_model = EyesModel(pretrained_model_eyes)
        self.face_grid_model = FaceGridModel()
        self.screen_features = screen_features
        if not screen_features:
            self.fc = nn.Sequential(
                nn.Linear(128+64+128, 128),
                nn.ReLU(inplace=True),
                nn.Linear(128, 64),
                nn.ReLU(inplace=True),
                nn.Linear(64, 32),
                nn.ReLU(inplace=True),
                nn.Linear(32, 2),
            )
        else:
            self.fc1 = nn.Sequential(
                nn.Linear(128+64+128, 128),
                nn.ReLU(inplace=True),
                nn.Linear(128, 13),
                nn.ReLU(inplace=True),
            )
            self.layer_norm = nn.LayerNorm(16)
            self.fc2 = nn.Linear(16, 2)
            

    def _preprocess(self, x):
        return x.to(torch.float32) * 2.0 - 1.0
        
    def forward(self, x_face, x_eye_l, x_eye_r, x_grid, x_screen = None):
        if self.screen_features and x_screen is None:
            raise Exception("You should pass screen features")
        if not self.screen_features and x_screen is not None:
            warnings.warn("Screen fearures won't be used")
        x_eyes = self.eyes_model(x_eye_l, x_eye_r)
        x_face = self.face_model(x_face)
        x_grid = self.face_grid_model(x_grid)
        x = torch.cat([x_eyes, x_face, x_grid], axis = 1)
        if not self.screen_features:
            x = self.fc(x)
        else:
            x = self.fc1(x)
            x = torch.cat([x, x_screen], axis = 1)
            x = self.layer_norm(x)
            x = self.fc2(x)
        return x

In [281]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

experiment = Experiment(
  api_key="4qtNKAjcucKnOrwC4pRvPaHRv",
  project_name="tweakle-gaze-calibration",
  workspace="kmisterios"
)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/kmisterios/tweakle-gaze-calibration/145784d0d37b474d92028585d653755f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     best_model_epoch [21] : (0, 3)
COMET INFO:     epoch [21]            : (0, 20)
COMET INFO:     lr [21]               : (1.2537500000000002e-05, 5.000000000000001e-05)
COMET INFO:     rmse_train [21]       : (2.130670816312247, 2.8134667618515072)
COMET INFO:     rmse_val [21]         : (0.6479468624619621, 0.7003415674279497)
COMET INFO:     rmse_val_hard [21]    : (0.7605066532391761, 0.844

In [282]:
experiment.set_name(f"{EXPERIMENT_NAME}")

In [283]:
pretrained_model_face = FaceMesh()
pretrained_model_face.load_weights("./weights/facemesh.pth")

model_path = "./weights/irislandmarks.pth"
pretrained_model_eyes = IrisLM()
weights = torch.load(model_path)
pretrained_model_eyes.load_state_dict(weights)

<All keys matched successfully>

In [284]:
CHECKPOINTS_PATH = "./checkpoints"
# EXPERIMENT_NAME_ORIG = "face_eyes_mask_more_layers_more_patience_weighted_loss_more_complexity_tune"
EXPERIMENT_NAME_ORIG = "calibration_unfreezed_guys_cleaned_weighted_loss_more_warmup_aug"

In [285]:
model = FineTuneModel(pretrained_model_face, pretrained_model_eyes, screen_features=False).to(device)
model.load_state_dict(torch.load(os.path.join(CHECKPOINTS_PATH, f"best_{EXPERIMENT_NAME_ORIG}.pt")))
model.train()

FineTuneModel(
  (face_model): FaceModel(
    (backbone): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
      (1): PReLU(num_parameters=16)
      (2): FaceMeshBlock(
        (convs): Sequential(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16)
          (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (act): PReLU(num_parameters=16)
      )
      (3): FaceMeshBlock(
        (convs): Sequential(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16)
          (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (act): PReLU(num_parameters=16)
      )
      (4): FaceMeshBlock(
        (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (convs): Sequential(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), groups=16)
          (1): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
   

In [286]:
# for param in model.parameters():
#     param.requires_grad = False

# for param in model.fc.parameters():
#     param.requires_grad = True

In [287]:
criterion = RMSELoss(weights = LOSS_WEIGHTS)
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY)
scheduler = ReduceLROnPlateau(optimizer, factor= REDUCE_FACTOR, patience=PATIENCE)

In [288]:
warmup = torch.optim.lr_scheduler.LinearLR(
    optimizer,
    start_factor=0.001,
    end_factor=1.0,
    total_iters=4,
)

In [ ]:
val_loss_min = np.inf
epoch_save = None

for epoch in range(NUM_EPOCHS):
    train_loss, train_mape = train(model, dataloader_train)
    val_loss, val_mape = eval(model, dataloader_val)
    val_loss_hard, val_mape_hard = eval(model, dataloader_val_hard)
    
    if epoch <= 4:
        warmup.step()
        current_lr = warmup.optimizer.param_groups[0]['lr']
    else:
        scheduler.step(val_loss_hard)
        current_lr = scheduler.optimizer.param_groups[0]['lr']
        
    if val_loss_min > val_loss_hard:
        val_loss_min = val_loss_hard
        torch.save(model.state_dict(), os.path.join(CHECKPOINTS_PATH, f"best_{EXPERIMENT_NAME}.pt"))
        epoch_save = epoch

    print()
    print(f'Epoch: {epoch}: Train loss: {round(train_loss, 3)}; Val loss: {round(val_loss, 3)};; Val loss hard: {round(val_loss_hard, 3)};')
    if epoch_save is not None:
        print(f'Best model saved on epoch {epoch_save}')
    experiment.log_metrics({
        "rmse_val": val_loss,
        "rmse_val_hard": val_loss_hard,
        "rmse_train": train_loss,
        "best_model_epoch": epoch_save,
        "train_mape": train_mape,
        "val_mape": val_mape,
        "val_mape_hard": val_mape_hard,
        "epoch": epoch,
        "lr": current_lr
    })
experiment.end()


Epoch: 0: Train loss: 2.715; Val loss: 0.677;; Val loss hard: 0.844;
Best model saved on epoch 0

Epoch: 1: Train loss: 2.628; Val loss: 0.676;; Val loss hard: 0.773;
Best model saved on epoch 1

Epoch: 2: Train loss: 2.562; Val loss: 0.701;; Val loss hard: 0.771;
Best model saved on epoch 2

Epoch: 3: Train loss: 2.493; Val loss: 0.685;; Val loss hard: 0.799;
Best model saved on epoch 2

Epoch: 4: Train loss: 2.457; Val loss: 0.662;; Val loss hard: 0.831;
Best model saved on epoch 2

Epoch: 5: Train loss: 2.352; Val loss: 0.662;; Val loss hard: 0.794;
Best model saved on epoch 2

Epoch: 6: Train loss: 2.261; Val loss: 0.659;; Val loss hard: 0.8;
Best model saved on epoch 2

Epoch: 7: Train loss: 2.189; Val loss: 0.659;; Val loss hard: 0.808;
Best model saved on epoch 2


In [ ]:
# experiment.end()

In [ ]:
model = FineTuneModel(pretrained_model_face, pretrained_model_eyes, screen_features=False).to(device)
model.load_state_dict(torch.load(os.path.join(CHECKPOINTS_PATH, f"best_{EXPERIMENT_NAME}.pt")))
# model.load_state_dict(torch.load(os.path.join(CHECKPOINTS_PATH, f"best_raw_images.pt")))
criterion = RMSELoss(weights = LOSS_WEIGHTS)
model.eval()

In [ ]:
# model.eval()

In [ ]:
dataset_test = GazeDetectionDataset(data = test_df, transform_list=trans_list, to_tensors=True, device=device, screen_features=False)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=0)

In [ ]:
from tqdm import tqdm

preds = []
labels_list = []
losses = 0
for i, data in tqdm(enumerate(dataloader_test), total = len(dataloader_test)):
    inputs, labels, inputs_eye_l, inputs_eye_r, inputs_mask = data['image'], data['coordinates'], \
                                                     data['eye_l'], data['eye_r'], data['face_mask']
    with torch.no_grad():
        outputs = model(inputs, inputs_eye_l, inputs_eye_r, inputs_mask)
    loss = criterion(outputs, labels)
    losses += loss.detach().cpu().item()
    pred = outputs.cpu().numpy()
    preds.append(pred)
    labels_list.append(labels.cpu().numpy())

print(f"Test loss: {round(losses / (i + 1), 3)}")

In [ ]:
print(f"Test loss: {round(losses / (i + 1), 3)}")

In [ ]:
preds = np.vstack(preds)
labels = np.vstack(labels_list)

mape_value = mape(labels, preds)
print(f"Test MAPE: {mape_value}")

test_df_copy = test_df.copy()

test_df_copy["pred_x"] = preds.T[0]
test_df_copy["pred_y"] = preds.T[1]

test_df_copy[['x_normalized', 'y_normalized', 'pred_x', 'pred_y']].tail(40)

In [ ]:
dataset_test = GazeDetectionDataset(data = test_diff_person, transform_list=trans_list, to_tensors=True, device=device, screen_features=False)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=0)

In [ ]:
from tqdm import tqdm

preds = []
labels_list = []
losses = 0
for i, data in tqdm(enumerate(dataloader_test), total = len(dataloader_test)):
    inputs, labels, inputs_eye_l, inputs_eye_r, inputs_mask = data['image'], data['coordinates'], \
                                                     data['eye_l'], data['eye_r'], data['face_mask']
    with torch.no_grad():
        outputs = model(inputs, inputs_eye_l, inputs_eye_r, inputs_mask)
    loss = criterion(outputs, labels)
    losses += loss.detach().cpu().item()
    pred = outputs.cpu().numpy()
    preds.append(pred)
    labels_list.append(labels.cpu().numpy())

print(f"Test loss: {round(losses / (i + 1), 3)}")

In [ ]:
preds = np.vstack(preds)
labels = np.vstack(labels_list)

mape_value = mape(labels, preds)
print(f"Test MAPE: {mape_value}")

test_df_copy = test_diff_person.copy()

test_df_copy["pred_x"] = preds.T[0]
test_df_copy["pred_y"] = preds.T[1]

test_df_copy[['x_normalized', 'y_normalized', 'pred_x', 'pred_y']].tail(40)

In [ ]:
test_df_copy.to_csv("./real_experiment/res_calibration_unfreezed_guys_cleaned_weighted_loss_more_warmup_aug_tune_tune_tune.csv", index = False)